In [1]:
from sklearn.preprocessing import label_binarize, LabelBinarizer
import numpy as np
from sklearn.utils.extmath import softmax
from sklearn.metrics import accuracy_score, log_loss

In [2]:
def initialize_pop(input_neuron, hidden_neuron, output_neuron, pop_size, low, high):
    pop = [dict() for i in range(pop_size)]
    
    for i in range(pop_size):
        wH = np.random.uniform(low, high, size = (hidden_neuron, input_neuron))
        wO = np.random.uniform(low, high, size = (output_neuron, hidden_neuron))
        
        bH = np.random.uniform(low, high, size = (hidden_neuron, 1))
        bO = np.random.uniform(low, high, size = (output_neuron, 1))
        
        pop[i] = {"wH": wH, "wO": wO, "bH": bH, "bO": bO}
    
    return pop

In [3]:
def compute_loss(curGen, pop_size, X, y):
    losses = [float() for i in range(pop_size)]
    acc = [float() for i in range(pop_size)]
    
    for i in range(pop_size):
        wH = curGen[i]["wH"]
        bH = curGen[i]["bH"]
        
        zH = np.dot(wH, X.T) + bH
        
        oH = np.maximum(zH, 0)
        
        wO = curGen[i]["wO"]
        bO = curGen[i]["bO"]
        
        zO = np.dot(wO, oH) + bO
        
        zO = zO.T
        
        zO = softmax(zO)
        
        y_pred = np.argmax(zO, axis = 1)
        y_pred = label_binarize(y_pred, classes = [0, 1])
        
        acc[i] = accuracy_score(y, y_pred)
        losses[i] = log_loss(y, zO)
        
    return losses, acc

In [4]:
def comp_fitness(losses, pop_size):
    fitness = [float() for x in range(pop_size)]
    
    invertloss = [1/x for x in losses]
    sum = np.sum(invertloss)
    
    for i in range(pop_size):
        fitness[i] = (invertloss[i] / sum) * 100
        
    return fitness

In [5]:
def roulette(fitness, pop_size):
    count = 0
    p1 = -1
    p2 = -1
    
    while count != 2:
        val = np.random.random_sample()*100
        
        cf = 0
        i = 0
        while cf < val and i < pop_size:
            cf += fitness[i]
            i += 1
            
        if p1 == -1:
            p1 = i
            count += 1
        
        elif p2 == -1 and p1 != i:
            p2 = i
            count += 1
    
    return (p1-1, p2-1)

In [6]:
def flatten(chromo):
    chromo_flattened = []
    for i in chromo:
        chromo_flattened = np.concatenate((chromo_flattened, chromo[i].flatten()))
    return chromo_flattened

In [7]:
def unflatten(chromo_flattened, iCtr, hCtr, oCtr):
    temp = np.split(chromo_flattened, [hCtr * iCtr])
    wHflat = temp[0]
    chromo_flattened = temp[1]
    
    temp = np.split(chromo_flattened, [oCtr * hCtr])
    wOflat = temp[0]
    chromo_flattened = temp[1]
    
    temp=np.split(chromo_flattened,[hCtr])
    bHflat=temp[0]
    chromo_flattened=temp[1]
    
    bOflat = chromo_flattened
    
    wH = np.array(wHflat).reshape(hCtr, iCtr)
    wO = np.array(wOflat).reshape(oCtr, hCtr)
    bH = np.array(bHflat).reshape(hCtr, 1)
    bO = np.array(bOflat).reshape(oCtr, 1)
    
    chromo = {"wH": wH, "wO": wO, "bH": bH, "bO": bO}
    
    return chromo

In [8]:
def flattenGen(curGen):
    curGenFlat = []
    
    for chromo in curGen:
        chromo_flat = flatten(chromo)
        curGenFlat.append(chromo_flat)
        
    return curGenFlat

In [9]:
def unFlattenGen(curGenFlat, iCtr, hCtr, oCtr):
    curGen = []
    
    for chromo_flat in curGenFlat:
        chromo = unflatten(chromo_flat, iCtr, hCtr, oCtr)
        curGen.append(chromo)
        
    return curGen    

In [10]:
def onepointcrossover(p1, p2, tot_genes):
    point = np.random.randint(1, tot_genes)
    
    off1 = np.concatenate((p2[0:point],p1[point:]))
    off2 = np.concatenate((p1[0:point],p2[point:]))
    
    return off1, off2

In [11]:
def onepointmutation(chromo, tot_genes, low, high, ProbOfMut):
    
    for i in range(tot_genes):
        num = np.random.rand()
        if num < ProbOfMut:
            chromo[i] = float(np.random.uniform(-1.0,1.0,(1,1))) 
    
    return chromo

In [12]:
def nextGen(curGenFlat, fitness, popLoss, pop_size, tot_genes, prob_cross, prob_mut, low, high):
    nextGenFlat = curGenFlat
    i = 0
    
    while i < pop_size:
        p1, p2 = roulette(fitness, pop_size)
        
        nextGenFlat[i] = curGenFlat[p1]
        nextGenFlat[i+1] = curGenFlat[p2]
        
        rand = np.random.rand()
        if rand < prob_cross:
            off1, off2 = onepointcrossover(curGenFlat[p1], curGenFlat[p2], tot_genes)
            nextGenFlat[i] = off1
            nextGenFlat[i+1] = off2
            
        nextGenFlat[i] = onepointmutation(nextGenFlat[i], tot_genes, low, high, prob_mut)
        nextGenFlat[i+1] = onepointmutation(nextGenFlat[i+1], tot_genes, low, high, prob_mut)
        
        i += 2
    
    return nextGenFlat

In [13]:
import pandas as pd
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [14]:
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [15]:
data = data.drop(["Age", "ZIP Code"], axis = 1)

In [16]:
X = data.drop(["Personal Loan"], axis = 1)
y = data.loc[:, ["Personal Loan"]]

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [18]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
input_neuron = X.shape[1]
hidden_neuron = 4
output_neuron = np.unique(y).shape[0]

encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [20]:
tot_genes = input_neuron * hidden_neuron + hidden_neuron * output_neuron + hidden_neuron + output_neuron

pop_size = 30
low, high = -2.0, 2.0
prob_cross = 0.8
prob_mut = 0.01
maxGen = 30

In [21]:
iniPop = initialize_pop(input_neuron, hidden_neuron, output_neuron, pop_size, low, high)
curGen = iniPop

In [22]:
for i in range(maxGen):
    
    loss, acc = compute_loss(curGen, pop_size, X_train, y_train)
    fitness = comp_fitness(loss, pop_size)
    
    curGenFlat = flattenGen(curGen)
    nextGenFlat = nextGen(curGenFlat, fitness, loss, pop_size, tot_genes, prob_cross, prob_mut, low, high)
    nextGenUnflat = unFlattenGen(nextGenFlat, input_neuron, hidden_neuron, output_neuron)
    
    curGen = nextGenUnflat

In [24]:
import torch
import torch.nn as nn

X_test = torch.Tensor(X_test)
X_test = X_test.to(torch.float32)

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=11, out_features=4)
        self.fc2 = nn.Linear(in_features=4, out_features=2)
        self.init_weights()

    def forward(self, x):
        out = torch.relu(self.fc1(x))
        out = self.fc2(out)
        return out

    def init_weights(self):
        with torch.no_grad():
              self.fc1.weight = nn.Parameter(torch.from_numpy(curGen[0]["wH"]))
              self.fc1.bias = nn.Parameter(torch.from_numpy(curGen[0]["bH"]).flatten())
              self.fc2.weight = nn.Parameter(torch.from_numpy(curGen[0]["wO"]))
              self.fc2.bias = nn.Parameter(torch.from_numpy(curGen[0]["bO"]).flatten())

model = Net()
model = model.float()

In [27]:
y_test_encoded = y_test.reshape(-1)
y_test_encoded = np.eye(2)[y_test.T]

In [28]:
y_test_encoded = torch.from_numpy(y_test.astype("float32"))

In [31]:
def accuracy(outputs, labels):
    outputs = outputs.numpy()
    output_list = []
    for i in outputs:
        if i[0] > i[1]:
            output_list.append(0)
        else:
            output_list.append(1)
    correct = (output_list == y_test).sum()
    return correct / len(labels)

model.eval()
test_acc = 0 
with torch.no_grad():
    outputs = model(X_test)
    test_acc += accuracy(outputs, y_test).item()
print(f"Test Accuracy: {test_acc/len(X_test):.3f}")

Test Accuracy: 0.915
